Ucitavanje biblioteka

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
Učitavanje i prikazivanje podataka iz csv datoteke.

In [5]:
podaci = pd.read_csv(r"C:\Users\Domagoj\Downloads\PripremaPodataka_ints.csv")
podaci[['Crash Year','Crash Month','Crash Day','Time of Day','Day of Week','City or Township','Crash: Intersection','Crash: Hit-and-Run','Lighting Conditions','Weather Conditions','Speed Limit at Crash Site','Imputed_Speed Limit at Crash Site','Worst Injury In Crash','Person Age','Imputed_Person Age','Person Gender','ADI Impute Column','Column 2']].describe()

,Crash Year,Crash Month,Crash Day,Day of Week,Crash: Intersection,Crash: Hit-and-Run,Lighting Conditions,Weather Conditions,Speed Limit at Crash Site,Worst Injury In Crash,Person Age,Person Gender
count,6809.000000,6809.000000,6809.000000,6809.000000,6809.000000,6809.000000,6809.000000,6809.000000,6367.000000,6809.000000,4333.000000,6809.000000
mean,2013.992216,6.764283,15.384197,3.916287,0.434278,0.368483,2.964606,1.166104,31.689964,1.652812,42.711285,0.685123
std,2.597471,3.467210,8.701025,1.934833,0.495698,0.482429,1.181804,0.682923,7.697713,1.135583,17.493884,0.464501
min,2010.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,5.000000,0.000000,4.000000,0.000000
25%,2012.000000,4.000000,8.000000,2.000000,0.000000,0.000000,2.000000,1.000000,25.000000,1.000000,28.000000,0.000000
50%,2014.000000,7.000000,15.000000,4.000000,0.000000,0.000000,2.000000,1.000000,30.000000,1.000000,41.000000,1.000000
75%,2016.000000,10.000000,23.000000,6.000000,1.000000,1.000000,4.000000,1.000000,35.000000,2.000000,56.000000,1.000000
max,2018.000000,12.000000,31.000000,7.000000,1.000000,1.000000,6.000000,8.000000,60.000000,4.000000,95.000000,1.000000


In [6]:
podaci.describe(include='object')

,Time of Day,City or Township,Imputed_Speed Limit at Crash Site,Imputed_Person Age,ADI Impute Column,Column 2
count,6809,6809,6809,6809,6809,6809
unique,25,43,454,86,6809,86
top,6:00 PM - 7:00 PM,Wayne County: Detroit,25,"42,755034665",[2014 10 13 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 ...,"[42,7550346649059]"
freq,535,4543,2233,2476,1,2476


informacija o podacima (tipovi, nedostajuće vrijednosti)

In [7]:
podaci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6809 entries, 0 to 6808
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Crash Year                         6809 non-null   int64  
 1   Crash Month                        6809 non-null   int64  
 2   Crash Day                          6809 non-null   int64  
 3   Time of Day                        6809 non-null   object 
 4   Day of Week                        6809 non-null   int64  
 5   City or Township                   6809 non-null   object 
 6   Crash: Intersection                6809 non-null   int64  
 7   Crash: Hit-and-Run                 6809 non-null   int64  
 8   Lighting Conditions                6809 non-null   int64  
 9   Weather Conditions                 6809 non-null   int64  
 10  Speed Limit at Crash Site          6367 non-null   float64
 11  Imputed_Speed Limit at Crash Site  6809 non-null   objec